# LIBRARIES

In [12]:
from pandas_datareader import data as pdr
import yfinance as yf
import pandas as pd
import numpy as np
import datetime
import collections
from collections import OrderedDict
from numpy import NaN, NAN,nan
import statistics
%run PART_2-FINAL.ipynb

# DATA ENGINEERING

In [13]:
all_data={}
for column in ['AAL','ALK','CEA','ZNH','VLRS','CPA','DAL','GOL','UAL']:
    all_data[column]=yf.download(column, start = '2014-06-02', end='2016-06-13')

luvdf=pd.read_csv('LUV.csv')
luvdf['Date'] = pd.to_datetime(luvdf['Date'])
luvdf.index = luvdf['Date']
luvdf = luvdf.drop(columns = ['Date'])
luvdf = luvdf.iloc[21:]
luvdf=luvdf.iloc[:-1]

dict1={ 'Date':np.array([k for k in all_data['AAL'].index])}
for column in ['AAL','ALK','CEA','ZNH','VLRS','CPA','DAL','GOL','UAL']:
    dict1[column]=np.array([k for k in all_data[column]['Adj Close']])
price_industry = pd.DataFrame(dict1) 
price_industry.index = price_industry['Date']
price_industry = price_industry.drop(columns = ['Date'])

price_return=price_industry.pct_change(periods = 1)
price_return=price_return.replace(to_replace=[NaN],value=0)



[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


# Trading System

In [14]:
class Trading_strategy:
    def __init__(self,ob):
        self.todays_date = datetime.datetime.now().date()
        self.index = pd.date_range(self.todays_date, periods=1)
        self.columns = ['AdjClose_luv','AdjClose_ind']
        self.df= pd.DataFrame(index=self.index, columns=self.columns)
        self.df=self.df.drop(self.df.index[0])
        
        self.signals=pd.DataFrame(index=self.df.index,columns=['ind_signal','luv_signal',"positions"])        
        self.ob1 = ob
        
    
    def process_tick(self, adujusted_price,industry_price,volume):
        self.todays_date+=datetime.timedelta(days=1)
        row = pd.Series({'AdjClose_luv':adujusted_price,"AdjClose_ind":industry_price},name=self.todays_date)
        self.df = self.df.append(row)
        
        row2=pd.Series({'ind_signal':0.0,'luv_signal':0.0,"positions":0.0},name=self.todays_date)
        self.signals = self.signals.append(row2)
        
       
        self.check_signal()

        if np.array(self.signals["positions"])[-1]==1:
             self.generate_buy_order(adujusted_price,volume)
        elif np.array(self.signals["positions"])[-1]==-1:
            self.generate_sell_order(adujusted_price,volume)
            
        
        
    def check_signal(self):
        # Create short simple moving average over the short window
        self.signals['ind_sma'] =np.array(self.df["AdjClose_ind"].rolling(window=2, min_periods=1, center=False).mean())
        self.signals['ind_lma'] = np.array(self.df["AdjClose_ind"].rolling(window=14, min_periods=1, center=False).mean())

        # Create short simple moving average over the short window
        self.signals['luv_sma'] =np.array(self.df['AdjClose_luv'].rolling(window=2, min_periods=1, center=False).mean())
        self.signals['luv_lma'] = np.array(self.df['AdjClose_luv'].rolling(window=14, min_periods=1, center=False).mean())
        
        if self.signals.shape[0]>1:
            self.signals['ind_signal'][2:] = np.where(self.signals['ind_sma'][2:] > self.signals['ind_lma'][2:], 1.0, 0.0)   
            for i in range(2,self.signals["luv_signal"].shape[0]):
                if self.signals['luv_sma'][i]>self.signals['luv_lma'][i]:
                    if any(self.signals['ind_signal'][i-10:i+1]):
                        self.signals['luv_signal'][i]=1
            self.signals['positions'] = self.signals['luv_signal'].diff() 
            
    def generate_buy_order(self,adjusted_price,volume):
        order={'price':adjusted_price, 'quantity':volume, 'exchange' :"NYSE", 'side' :"BID"}
        self.ob1.process_tick(order)
        print("BUY",adjusted_price,self.todays_date)
    def generate_sell_order(self,adjusted_price,volume):
        order={'price':adjusted_price, 'quantity':volume, 'exchange' :"NYSE", 'side' :"OFFER"}
        self.ob1.process_tick(order)
        print("SELL",adjusted_price,self.todays_date)  
        
        

# EXECUTION

In [15]:
ob=Orderbook()
ts_new=Trading_strategy(ob)
list1=luvdf['Adj Close'].values
list2=price_industry.mean(axis=1)
list3=luvdf['Volume'].values

for i in range(len(list1)):
    ts_new.process_tick(list1[i],list2[i],list3[i])
    
print("******************************************************************************************")
print("orderbook")
ob.display_top_of_book()

$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$
BUY 14.544648 2020-11-07
SELL 14.073248000000001 2020-11-09
BUY 14.397335 2020-11-14
SELL 15.467806 2020-11-19
BUY 15.320494 2020-11-21
SELL 15.448164000000002 2020-12-03
BUY 14.780348000000002 2020-12-27
SELL 13.474178 2020-12-30
BUY 13.552744 2021-01-02
SELL 13.849301 2021-01-14
BUY 9.348278 2021-02-22
SELL 8.873445 2021-02-23
BUY 8.804198 2021-03-02
SELL 9.19 2021-03-14
BUY 6.73 2021-03-29
SELL 6.73 2021-04-08
BUY 6.25 2021-04-28
SELL 5.92 2021-04-30
BUY 6.26 2021-05-03
SELL 6.11 2021-05-11
BUY 6.35 2021-05-20
SELL 5.81 2021-05-22
BUY 5.43 2021-06-03
SELL 5.13 2021-06-05
BUY 5.22 2021-06-07
SELL 6.26 2021-06-23
BUY 5.33 2021-07-20
SELL 5.23 2021-07-21
BUY 5.29 2021-07-23
SELL 5.36 2021-07-28
BUY 5.39 2021-07-29
SELL 5.48 2021-08-08
BUY 3.57 2021-09-20
SELL 3.08 2021-09-28
BUY 3.61 2021-10-01
SELL 3.41 2021-10-07
BUY 3.39 2021-10-16
SELL 3.42 2021-10-27
BUY 3.45 2021-10-28
SELL 3.33 2021-